# Transaction Implementation
- Environment used: conda create -n blockchain pip python scipy cryptography
- cryptography version 3.4.7, python version 3.9.5
- Corresponds to script "Transaction.py"

# Class Definition For Blockchain Implementation

Includes the following:
- Definition class Transaction class and logic
- Utilizes methods to create inputs, outputs, require_signatures, and signatures objects
- Has **is_valid()** method which checks that are signatures received, and untampered. 
- Also check non-negativity of the amounts and that total amounts received is greater than sent. 


In [1]:
# importing
# from cryptography.hazmat.primitives.asymmetric import rsa
# from cryptography.hazmat.primitives import serialization
# from cryptography.hazmat.primitives import hashes
# from cryptography.hazmat.primitives.asymmetric import padding
# from cryptography.exceptions import InvalidSignature

import signatures

class Tx:
    inputs = None #input addresses
    outputs = None #output addresses and amounts
    sigs = None # list of signatures
    reqd = None # required signatures that are not inputs

    def __init__(self):
        self.inputs = []
        self.outputs = []
        self.sigs = []
        self.reqd = []
    def add_input(self, from_address, amount):
        '''takes a sending public_key and amount'''
        self.inputs.append((from_address,amount))

    def add_output(self, to_address, amount):
        '''takes a receiving public_key and amount'''
        self.outputs.append((to_address,amount))

    def add_required(self, address): # public_key is the address input here
        '''takes a required third-party public_key'''
        self.reqd.append(address)

    def sign(self, private_key):
        '''The signing user, signs using private_key creating signature'''
        message = self.__gather()
        new_sig = signatures.sign(message, private_key) # returns signed message
        self.sigs.append(new_sig)

    def __gather(self):
        '''internally used method, not callable. Used for digitally signed message'''
        data = []
        data.append(self.inputs)
        data.append(self.outputs)
        data.append(self.reqd)
        return(data)

    def is_valid(self):
        '''
        Validates all signatures using the data and public keys
        Validates that no negative transactions
        Validate that total amount in greater than total amount out
        Validate tampering not occur
        '''
        total_in = 0
        total_out = 0
        message = self.__gather()
        # verifying all input signatures, have been signed/received
        for public_key, amount in self.inputs:
            found = False
            for s in self.sigs:
                if signatures.verify(message, s, public_key):
                    found = True
            if not found:
                return False
            if amount < 0: #this checks for only positive input amounts
                return False
            total_in = total_in + amount
        for public_key in self.reqd:
            found = False
            for s in self.sigs:
                if signatures.verify(message, s, public_key):
                    found = True
            if not found:
                return False
        for addr, amount in self.outputs: #this checks for only negative input amounts
            if amount < 0:
                return False
            total_out = total_out + amount
        if total_out > total_in: #total in not less than total out
            return False
        return True


# Testing

In [2]:
# Making public and private keys for testing
pr1, pu1 = signatures.generate_keys()
pr2, pu2 = signatures.generate_keys()
pr3, pu3 = signatures.generate_keys()
pr4, pu4 = signatures.generate_keys()

In [3]:
Tx1 = Tx()
Tx1.add_input(pu1,1)
Tx1.add_output(pu2,1) #sending coin to pu2
Tx1.sign(pr1)
Tx1.is_valid()

True

In [4]:
Tx2 = Tx()
Tx2.add_input(pu1,2)
Tx2.add_output(pu2,1)
Tx2.add_output(pu3,1)
Tx2.sign(pr1)
Tx2.is_valid()

True

In [5]:
Tx3 = Tx()
Tx3.add_input(pu3,1.2) # shouldnt have more output than input.
Tx3.add_output(pu1,1.1) #mining and mining rewards, transaction fee usually goes to miner.
Tx3.sign(pr3)
Tx3.is_valid()

True